In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__valeurs_de_reference WHERE categorie_1='nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,categorie_1,categorie_2,categorie_3,limite_qualite,limite_qualite_unite,valeur_sanitaire_1,valeur_sanitaire_1_unite,valeur_sanitaire_2,valeur_sanitaire_2_unite,nb_rows
NO3,nitrate,None,None,50.0,mg/L,None,None,None,None,1
NO2,nitrate,None,None,0.5,mg/L,None,None,None,None,1
NO3_NO2,nitrate,None,None,1.0,None,None,None,None,None,1


In [3]:
%%sql --save int__resultats_nitrites_udi_annuel
WITH
prels AS (
    SELECT
        de_partition AS annee,
        cdreseau,
        referenceprel,
        datetimeprel,
        COUNT(DISTINCT cdparametresiseeaux) AS nb_parametres,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN valtraduite
                ELSE 0
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN valtraduite
                ELSE 0
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN valtraduite
                ELSE 0
            END
        ) AS valtraduite_no3_no2
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
    GROUP BY
        annee,
        cdreseau,
        referenceprel,
        datetimeprel,
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
        int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
)

SELECT
    prels.cdreseau,
    prels.annee,
    'nitrate' AS categorie,
    'bilan_annuel_' || prels.annee AS periode,
    COUNT(
        DISTINCT
        CASE
            WHEN
                prels.valtraduite_no3 >= valeur_ref.limite_qualite_no3
                OR prels.valtraduite_no2 >= valeur_ref.limite_qualite_no2
                OR prels.valtraduite_no3_no2
                >= valeur_ref.limite_qualite_no3_no2
                OR prels.valtraduite_no3 / 50 + prels.valtraduite_no2 / 3
                >= valeur_ref.limite_qualite_no3_no2
                THEN prels.referenceprel
        END
    ) AS nb_depassements,
    COUNT(DISTINCT prels.referenceprel) AS nb_prelevements,
    (
        COUNT(
            DISTINCT
            CASE
                WHEN
                    prels.valtraduite_no3 >= valeur_ref.limite_qualite_no3
                    OR prels.valtraduite_no2 >= valeur_ref.limite_qualite_no2
                    OR prels.valtraduite_no3_no2
                    >= valeur_ref.limite_qualite_no3_no2
                    OR prels.valtraduite_no3 / 50 + prels.valtraduite_no2 / 3
                    >= valeur_ref.limite_qualite_no3_no2
                    THEN prels.referenceprel
            END
        )::float
        /
        COUNT(DISTINCT prels.referenceprel)::float
    ) AS ratio_depassements

FROM prels
CROSS JOIN valeur_ref
GROUP BY prels.cdreseau, prels.annee


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,annee,categorie,periode,nb_depassements,nb_prelevements,ratio_depassements
010000144,2024,nitrate,bilan_annuel_2024,0,9,0.0
032000202,2023,nitrate,bilan_annuel_2023,0,16,0.0
088001565,2020,nitrate,bilan_annuel_2020,0,4,0.0
021000787,2023,nitrate,bilan_annuel_2023,0,3,0.0
017000374,2022,nitrate,bilan_annuel_2022,0,23,0.0
073000507,2023,nitrate,bilan_annuel_2023,0,3,0.0
069000437,2021,nitrate,bilan_annuel_2021,0,6,0.0
067001330,2020,nitrate,bilan_annuel_2020,0,8,0.0
089003503,2020,nitrate,bilan_annuel_2020,3,12,0.25
064000416,2023,nitrate,bilan_annuel_2023,0,4,0.0


# Check 

In [7]:
%%sql --with int__resultats_nitrites_udi_annuel 
SELECT
    'annuel' AS periode,
    cdreseau,
    categorie,
    '' AS resultat,
    nb_depassements,
    nb_prelevements,
    ratio_depassements
FROM
    int__resultats_nitrites_udi_annuel
WHERE
    (
        cdreseau = '092003070'
        AND categorie = 'nitrate'
        AND annee = '2024'
        AND (nb_prelevements != 806 
        OR  nb_depassements != 0  )
    )
    OR
    (
        cdreseau = '071001155'
        AND categorie = 'nitrate'
        AND annee = '2023'
        AND (nb_prelevements != 1 
        OR  nb_depassements != 0  )
    )
    OR
    (
        cdreseau = '036000670'
        AND categorie = 'nitrate'
        AND annee = '2024'
        AND (nb_prelevements != 27 
        OR  nb_depassements != 27  )
    )
    OR
    (
        cdreseau = '092003070'
        AND categorie = 'nitrate'
        AND annee IN (2020, 2021, 2022, 2023, 2024, 2025) 
        AND nb_depassements != 0 
    )
    OR
    (
        cdreseau = '089003503'
        AND categorie = 'nitrate'
        AND annee = '2020'
        AND (nb_prelevements != 12 
        OR  nb_depassements != 3  )
    )

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

periode,cdreseau,categorie,resultat,nb_depassements,nb_prelevements,ratio_depassements
